https://spacy.io/usage/visualizers

https://explosion.ai/blog/deep-learning-formula-nlp
- video 
https://www.youtube.com/watch?v=l4scwf8KeIA

In [1]:
import spacy

In [2]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
doc = nlp(u"he works at Google.")
spacy.displacy.serve(doc, style='ent')


    Serving on port 5000...
    Using the 'ent' visualizer



127.0.0.1 - - [22/Oct/2018 15:19:52] "GET / HTTP/1.1" 200 804
127.0.0.1 - - [22/Oct/2018 15:19:53] "GET /favicon.ico HTTP/1.1" 200 804
